In [ ]:
!conda install -c conda-forge pillow -y
!conda install -c conda-forge pydicom -y
!conda install -c conda-forge gdcm -y
!pip install pylibjpeg pylibjpeg-libjpeg

In [ ]:
!pip3 install python-gdcm

In [ ]:
import os
import shutil
import pydicom
import pylibjpeg
import gdcm

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
import numpy as np
import cv2
from PIL import Image
import pandas as pd
import gc
from tqdm import tqdm



import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Reshape, Permute, Activation, Input, \
    add, multiply,Flatten, ReLU, Dense, AveragePooling2D, MaxPool2D, GlobalMaxPooling2D, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.layers import concatenate, Dropout
from tensorflow.keras.models import Model

In [ ]:
#not_normal
df = pd.read_csv("/kaggle/input/rsna-str-pulmonary-embolism-detection/train.csv")
df = df.loc[df["pe_present_on_image"]==1,:].reset_index(drop=True)
print(len(df))
df.tail()

In [ ]:
#normal
df1 = pd.read_csv("/kaggle/input/rsna-str-pulmonary-embolism-detection/train.csv")
df1 = df1.loc[(df1["pe_present_on_image"] == 0) & (df1["negative_exam_for_pe"] == 1) ,:].reset_index(drop=True)
print(len(df1))
df1.head()

In [ ]:
def show_difference():
    plt.subplots(figsize=(20, 20))
    for i in range(6):
        r_ind = np.random.randint(0,len(df)-1)
        dcm = pydicom.dcmread("/kaggle/input/rsna-str-pulmonary-embolism-detection/train/"+df.loc[r_ind,'StudyInstanceUID']+'/'+df.loc[r_ind,'SeriesInstanceUID']+'/'+df.loc[r_ind,'SOPInstanceUID']+'.dcm')
        dcm.PhotometricInterpretation = 'YBR_FULL'
        dc_image=dcm.pixel_array.astype('float32')
        im = Image.fromarray(dc_image).convert('HSV').resize((256,256))
        plt.subplot(2, 6, i+1)
        plt.imshow(im)
        plt.title('Disease')
        plt.axis('off')

        r_ind = np.random.randint(0,len(df1)-1)
        dcm = pydicom.dcmread("/kaggle/input/rsna-str-pulmonary-embolism-detection/train/"+df1.loc[r_ind,'StudyInstanceUID']+'/'+df1.loc[r_ind,'SeriesInstanceUID']+'/'+df1.loc[r_ind,'SOPInstanceUID']+'.dcm')
        dcm.PhotometricInterpretation = 'YBR_FULL'
        dc_image = dcm.pixel_array.astype('float32')
        im = Image.fromarray(dc_image).convert('HSV').resize((256,256))
        plt.subplot(2, 6, i+7)
        plt.imshow(im)
        plt.title('Healthy')
        plt.axis('off')

In [ ]:
show_difference()

In [ ]:
def show_distribution():
    images = []
    for i in tqdm(range(30)):
        dcm = pydicom.dcmread("/kaggle/input/rsna-str-pulmonary-embolism-detection/train/"+df.loc[i,'StudyInstanceUID']+'/'+df.loc[i,'SeriesInstanceUID']+'/'+df.loc[i,'SOPInstanceUID']+'.dcm')
        dcm.PhotometricInterpretation = 'YBR_FULL'
        dc_image=dcm.pixel_array.astype('float32')
        HU_image = dcm.RescaleIntercept + dcm.RescaleSlope * dc_image
        images.append(HU_image)

    images = np.array(images)

    sns.displot(images.flatten())

In [ ]:
show_distribution()

In [ ]:
def show_rescaling(MINI = -500, MAXI = 500):
    plt.subplots(figsize=(20, 20))
    for i in range(6):
        r_ind = np.random.randint(0,len(df)-1)
        r_ind = i
        dcm = pydicom.dcmread("/kaggle/input/rsna-str-pulmonary-embolism-detection/train/"+df.loc[r_ind,'StudyInstanceUID']+'/'+df.loc[r_ind,'SeriesInstanceUID']+'/'+df.loc[r_ind,'SOPInstanceUID']+'.dcm')
        dcm.PhotometricInterpretation = 'YBR_FULL'
        dc_image = dcm.pixel_array.astype('float32')
        HU_image = dcm.RescaleIntercept + dcm.RescaleSlope * dc_image
        
        
        plt.subplot(3, 6, i+1)
        plt.imshow(HU_image)
        plt.title('Original')
        plt.axis('off')
        
        scaled_image = np.maximum(HU_image, MINI)
        
        plt.subplot(3, 6, i+7)
        plt.imshow(scaled_image)
        plt.title('Min_cropped')
        plt.axis('off')

        
        scaled_image = np.minimum(scaled_image, MAXI)
        plt.subplot(3, 6, i+13)
        plt.imshow(scaled_image)
        plt.title('Max_cropped')
        plt.axis('off')

In [ ]:
show_rescaling(MINI = -370, MAXI = 430)

In [ ]:
#define window
MINI = -370
MAXI = 430

In [ ]:
def dcm_to_im(dcm):
    dcm.PhotometricInterpretation = 'YBR_FULL'
    dc_image=dcm.pixel_array.astype('float32')
    HU_image = dcm.RescaleIntercept + dcm.RescaleSlope * dc_image
    scaled_image = np.maximum(HU_image, MINI)
    scaled_image = np.minimum(scaled_image, MAXI)
    im = Image.fromarray(scaled_image).convert('RGB').resize((256,256))
    return im

In [ ]:
!mkdir /kaggle/Pulmonary_dataset/
!mkdir /kaggle/Pulmonary_dataset/RSNA/
!mkdir /kaggle/Pulmonary_dataset/RSNA/healthy/
!mkdir /kaggle/Pulmonary_dataset/RSNA/disease/

In [ ]:
def process_and_transfer_data_all():
    for i in tqdm(range(len(df))):
        dcm = pydicom.dcmread("/kaggle/input/rsna-str-pulmonary-embolism-detection/train/"+df.loc[i,'StudyInstanceUID']+'/'+df.loc[i,'SeriesInstanceUID']+'/'+df.loc[i,'SOPInstanceUID']+'.dcm')
        im = dcm_to_im(dcm)
        im.save("/kaggle/Pulmonary_dataset/RSNA/disease/"+str(i)+".jpg")
    
    for i in tqdm(range(2*len(df))):
        dcm = pydicom.dcmread("/kaggle/input/rsna-str-pulmonary-embolism-detection/train/"+df1.loc[i,'StudyInstanceUID']+'/'+df1.loc[i,'SeriesInstanceUID']+'/'+df1.loc[i,'SOPInstanceUID']+'.dcm')
        im = dcm_to_im(dcm)
        im.save("/kaggle/Pulmonary_dataset/RSNA/healthy/"+str(i)+".jpg")

In [ ]:
process_and_transfer_data_all()

In [ ]:
!mkdir /root/.kaggle

In [ ]:
import json
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("API_code")

aDict = {"username":"diorzhey","key":secret_value_0}

jsonString = json.dumps(aDict)
jsonFile = open("/root/.kaggle/kaggle.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [ ]:
import json

aDict = {  
         "title": "Pulmonary_dataset", 
         "id": "diorzhey/Pulmonary-dataset", 
         "licenses": [{"name": "CC0-1.0"}]
}
jsonString = json.dumps(aDict)
jsonFile = open("/kaggle/Pulmonary_dataset/dataset-metadata.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
%%capture
!zip -r /kaggle/Pulmonary_dataset/RSNA.zip /kaggle/Pulmonary_dataset/RSNA

In [ ]:
!rm -r /kaggle/Pulmonary_dataset/RSNA

In [ ]:
!ls ../Pulmonary_dataset/

In [ ]:
!kaggle datasets create -p /kaggle/Pulmonary_dataset --dir-mode zip